In [1]:
import numpy as np 

from aiida import load_profile
from aiida.orm import load_node
from aiida_vasp_ext_mlauer.util.structures import gen_aiida_structure

from aiida.plugins import DataFactory

load_profile('lauerm-test')

Profile<uuid='34741e4bf9c34bd7b9ae17bef7c5da2e' name='lauerm-test'>

In [2]:
def generate_wz_structure(alat, clat, species):

    lattice = np.array(
        [[1/2, -np.sqrt(3)/2, 0],
         [1/2,  np.sqrt(3)/2, 0],
         [  0,             0, clat/alat]]
    )

    basis = np.array(
        [[1/3, 2/3, 0.0],
         [2/3, 1/3, 0.5],
         [1/3, 2/3, 0.375],
         [2/3, 1/3, 0.875]]
    )

    aiida_str = gen_aiida_structure(alat, lattice, species, basis)
    aiida_str.store()

    return aiida_str

In [3]:
# STRUCTURE_PKs = [
#     688, # wz-AlN
#     689, # wz-ScN
#     690, # wz-GaN
#     691, # wz-InN
# ]

STRUCTURE_PKs = [None, None, None, None]

INIT_LAT_PARA = [
    (3.112, 4.982), # AlN - Vurgaftman
    (3.715,	4.475), # ScN - https://pmc.ncbi.nlm.nih.gov/articles/PMC12250753/
    (3.189, 5.185), # GaN - Vurgaftman
    (3.545, 5.703), # InN - Vurgaftman
]

SPECIES = [
    ["Al", "Al", "N", "N"],
    ["Sc", "Sc", "N", "N"],
    ["Ga", "Ga", "N", "N"],
    ["In", "In", "N", "N"],
]

In [4]:

for idx in range(4):
    species = SPECIES[idx]
    alat, clat = INIT_LAT_PARA[idx]
    if STRUCTURE_PKs[idx] is None:
        structure = generate_wz_structure(alat, clat, species)
    
    else:
        structure = load_node(STRUCTURE_PKs[idx])
    
    print(f"Hexagonal wz-{"".join(np.unique(species))} is stored at {structure.pk}")

    pmg_str = structure.get_pymatgen()
    pmg_str.to_file(f'../test/{"".join(np.unique(species))}.vasp', fmt='poscar')

Hexagonal wz-AlN is stored at 5827
Hexagonal wz-NSc is stored at 5828
Hexagonal wz-GaN is stored at 5829
Hexagonal wz-InN is stored at 5830
